# Stage 3: Multimedia Generation
This notebook generates audio (TTS) and visual assets (AI Images) based on the script.

In [1]:
import json
import os
import asyncio
import edge_tts
import requests
from mutagen.mp3 import MP3

# Ensure assets directory
ASSETS_DIR = "../assets"
os.makedirs(ASSETS_DIR, exist_ok=True)

async def generate_audio(text, output_file, voice="en-US-GuyNeural"):
    communicate = edge_tts.Communicate(text, voice)
    await communicate.save(output_file)

def get_audio_duration(file_path):
    audio = MP3(file_path)
    return audio.info.length

def generate_image(prompt, output_file):
    # Using Pollinations.ai for free AI images
    url = f"https://image.pollinations.ai/prompt/{prompt}?width=1080&height=1920&nologo=true"
    try:
        response = requests.get(url, timeout=30)
        if response.status_code == 200:
            with open(output_file, 'wb') as f:
                f.write(response.content)
            return True
    except Exception as e:
        print(f"Error generating image: {e}")
    return False

async def process_script():
    input_path = "../data/script.json"
    if not os.path.exists(input_path):
        print("Script file not found!")
        return
    
    with open(input_path, "r", encoding="utf-8") as f:
        scripts = json.load(f)
    
    if not scripts:
        print("No scripts found.")
        return
    
    # Process the first script
    script = scripts[0]
    segments = script.get("segments", [])
    
    for i, segment in enumerate(segments):
        print(f"Processing Segment {i}...")
        text = segment.get("text", "")
        prompt = segment.get("image_prompt", "Abstract visualization")
        
        # Audio
        audio_path = os.path.join(ASSETS_DIR, f"{i}_audio.mp3")
        await generate_audio(text, audio_path)
        duration = get_audio_duration(audio_path)
        print(f"  Audio generated: {duration:.2f}s")
        
        # Visual
        # Enforce vertical aspect ratio prompt if needed, but URL params handle it
        image_path = os.path.join(ASSETS_DIR, f"{i}_visual.jpg")
        success = generate_image(prompt, image_path)
        if success:
             print(f"  Image generated.")
        else:
             print(f"  Image failed.")
             
if __name__ == "__main__":
    # Jupyter runs an event loop, so we use loop.run_until_complete if outside, 
    # but inside jupyter we usually await directly or use a helper.
    # Since we are running via nbconvert, asyncio.run() works if there is no other loop,
    # OR we use the existing loop.
    try:
        loop = asyncio.get_event_loop()
        if loop.is_running():
             loop.create_task(process_script())
        else:
             loop.run_until_complete(process_script())
    except RuntimeError:
        asyncio.run(process_script())